In [1]:
BENCHMARK_TOOL = 'NVPROF' # or 'NCU'

import subprocess
import numpy as np
import pandas as pd
import io
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm 

In [2]:
KERNELS = [
    "rgb2hsv", 
    "rgb2hsv_MinimuDivergence",
    "rgb2hsv_CoordinatedOutputs",
    "histogram",
    "histogram_WithSharedMemory",
    "histogram_WithSharedMemoryAndHardcodedSize",
    "histogram_WithMinilmumCalculationDepencies",
    "repart",
    "repart_WithSharedMemory",
    "repart_WithSharedMemoryAndHardcodedSize",
    "equalization",
    "equalization_ConstantCoefficient",
    "hsv2rgb",
    "hsv2rgb_MinimuDivergence"
]

In [3]:
def read_nvprof(output: str) -> pd.DataFrame:
    csv = output.split('\\n')
    columns = csv[3].split('\",\"')
    data = csv[4:]

    return pd.read_csv(io.StringIO('\n'.join(data)), names=columns)

    
def read_ncu(output: str) -> pd.DataFrame:
    csv = output.split('\\n')
    columns = csv[2].split('\",\"')
    data = csv[4:]

    return pd.read_csv(io.StringIO('\n'.join(data)), names=columns)

def benchmark_kernel(exe_path: str, *args) -> str :
    if BENCHMARK_TOOL == 'NVPROF':
        return str(subprocess.check_output(
            ['nvprof', '--print-gpu-trace', '--csv', exe_path, *args], 
            stderr=subprocess.STDOUT
        ))
    elif BENCHMARK_TOOL == 'NCU':
        return str(subprocess.check_output(
            ['ncu','--csv', exe_path, *args], 
            stderr=subprocess.STDOUT, shell=True
        ))

def read_benchmark(output: str) -> pd.DataFrame:
    if BENCHMARK_TOOL == 'NVPROF':
        return read_nvprof(output)
    elif BENCHMARK_TOOL == 'NCU':
        return read_ncu(output)

def get_kernel_duration(df: pd.DataFrame, kernelNum=0) -> float:
    if BENCHMARK_TOOL == 'NVPROF':
        # conversion en ms 
        if df.loc[0, 'Duration'] == "ms":
            return float(df.loc[kernelNum + 1, 'Duration']) 
        if df.loc[0, 'Duration'] == "s":
            return float(df.loc[kernelNum + 1, 'Duration']) * 1e3
        # probablement en micro seconde
        return float(df.loc[kernelNum + 1, 'Duration']) * 1e-3
    elif BENCHMARK_TOOL == 'NCU':
        return float(df[df['Metric Name'] == 'Duration']['Metric Value'].to_numpy().item().replace(',','')) * 1e-6

In [4]:
# Renvoie les durée moyennes en fonction de variation de tailles de l'axe X des blocs pour un certain kernel
def evaluateBlocSize(kernelType, nTry = 10):  
    durations = []
    sizes = []
    avg = 0
    xBlockSize, yBlockSize = 1, 1
    
    while xBlockSize <= 1024:
        for _ in range(nTry) :         
            avg += get_kernel_duration(read_benchmark(
                benchmark_kernel("./hist", 
                    "-f", "./img/chateau.png", 
                    "-b", str(kernelType), 
                    "-d", str(xBlockSize), str(yBlockSize)
                )
            ), kernelType)

        avg /= nTry
        durations.append(avg)    
        sizes.append(xBlockSize)
        xBlockSize *= 2
    return durations, sizes

In [5]:
def showHistogramDuration(durations, sizes, title, xLabel, yLabel):
    plt.title(title)  
    plt.xlabel(xLabel) 
    plt.ylabel(yLabel) 
    plt.bar(np.array(sizes).astype(str), np.array(durations), width=.8) 
    plt.show()

def showHistogramOfKernels(kernels, nTry = 10):  
    i = 0
    for k in kernels: 
        durations, sizes = evaluateBlocSize(i, nTry)
        showHistogramDuration(durations, sizes, k, "taille de bloc", "ms") 
        i+=1
 
print("Évaluation des kernels suivant la variation de la taille des blocs sur l'axe X") 
#showHistogramOfKernels(KERNELS, 5)

Évaluation des kernels suivant la variation de la taille des blocs sur l'axe X


In [6]:
def evaluate2DBlocSize(kernelType, nTry = 10):  
    durations = []
    sizes = [] 
    xBlockSize = 1

    while xBlockSize <= 1024:
        yBlockSize = 1
        durations.append([])
        while yBlockSize <= 1024:
            if (xBlockSize * yBlockSize) > 1024:
                durations[-1].append(np.nan)    
                yBlockSize *= 2
                continue
            
            avg = 0
            for _ in range(nTry) :         
                avg += get_kernel_duration(read_benchmark(
                    benchmark_kernel("./hist", 
                        "-f", "./img/chateau.png", 
                        "-b", str(kernelType), 
                        "-d", str(xBlockSize), str(yBlockSize)
                    )
                ), kernelType)

            avg /= nTry
            durations[-1].append(avg)   
            yBlockSize *= 2
        sizes.append(xBlockSize)
         
        xBlockSize *= 2
    return durations, sizes
 

In [17]:
print("Variation de la taille des blocs rgb to hsv")
durations, sizes = evaluate2DBlocSize(0, 1)   
print(durations)
print(sizes)

Variation de la taille des blocs rgb to hsv
[[0.161152, 0.158944, 0.158816, 0.158496, 0.15904, 0.1584, 0.157121, 0.159072, 0.157504, 0.156896, 0.193408], [0.156993, 0.159903, 0.164896, 0.16032, 0.159103, 0.163104, 0.15744, 0.181888, 0.161056, 0.15776, nan], [0.157216, 0.158176, 0.158496, 0.157184, 0.159296, 0.160288, 0.158401, 0.166496, 0.159521, nan, nan], [0.160448, 0.157505, 0.158048, 0.159872, 0.160992, 0.158305, 0.163872, 0.157921, nan, nan, nan], [0.158816, 0.155841, 0.159712, 0.159264, 0.15776, 0.176737, 0.176192, nan, nan, nan, nan], [0.15584, 0.157536, 0.160576, 0.163712, 0.158241, 0.158625, nan, nan, nan, nan, nan], [0.157697, 0.157696, 0.15712, 0.157024, 0.15808, nan, nan, nan, nan, nan, nan], [0.158945, 0.15792, 0.158976, 0.180288, nan, nan, nan, nan, nan, nan, nan], [0.156384, 0.159552, 0.15872, nan, nan, nan, nan, nan, nan, nan, nan], [0.156704, 0.158977, nan, nan, nan, nan, nan, nan, nan, nan, nan], [0.158208, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]
[1, 2, 4, 

In [1]:
"""
def showHistogramDuration3D_1(durations, sizesX, sizesY, xLabel, yLabel): 
    # get colormap
    ncolors = 256
    color_array = plt.get_cmap('spring')(range(ncolors))

    # change alpha values
    color_array[:,-1] = [0.5] * 256

    # create a colormap object
    map_object = LinearSegmentedColormap.from_list(name='my_color',colors=color_array)

    # register this new colormap with matplotlib
    plt.register_cmap(cmap=map_object)

    fig = plt.figure()
    ax.set_title("test")
    ax = fig.add_subplot(projection='3d') 
    X, Y = np.meshgrid(sizesX, sizesY)   
    surf = ax.plot_surface(X, Y, np.array(durations), rstride=1, cstride=1, cmap=cm.coolwarm,
        linewidth=0, antialiased=False, vmin=0, vmax=10)

    
    # Customize the z axis.  
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.03f'))
    
    ax.legend() 
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel) 
    ax.set_zlabel("durée (en ms)")
 
    # Add a color bar which maps values to colors.evaluate2DBlocSize(4, 1) 
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.show()
    
def showHistogramDuration3D_2(durations, sizesX, sizesY, xLabel, yLabel): 
    # get colormap
    ncolors = 256
    color_array = plt.get_cmap('spring')(range(ncolors))

    # change alpha values
    color_array[:,-1] = [0.5] * 256

    # create a colormap object
    map_object = LinearSegmentedColormap.from_list(name='my_color',colors=color_array)

    # register this new colormap with matplotlib
    plt.register_cmap(cmap=map_object)

    fig = plt.figure()
    ax = fig.add_subplot(projection='3d') 
    X, Y = np.meshgrid(sizesX, sizesY)  

    dx = dy = 0.5 * np.ones_like(durations)
    dz = hist.ravel() 
    ax.bar3d(X.ravel(), Y.ravel(), durations, xLabel, dy, dz, zsort='average')
     
    
    # Customize the z axis.  
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.03f'))
    
    ax.legend() 
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel) 
    ax.set_zlabel("durée (en ms)")
 
    # Add a color bar which maps values to colors.evaluate2DBlocSize(4, 1)  
    plt.show()
"""

def getBestParameters(durations, sizesX, sizesY):
    m = durations[0][0]
    x = 1
    y = 1

    for i in range(len(durations)):
        for j in range(len(durations[i])):  
            if (durations[i][j] < m and durations[i][j] != np.nan):
                m = durations[i][j]
                x = sizesX[i] 
                y = sizesY[j] 

    return m, x, y 

def showHistogramDuration3DHist(durations, sizesX, sizesY, title, xLabel, yLabel): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title("Temps d'éxecution du kernel: " + title)
 
    xpos, ypos = np.meshgrid(sizesX, sizesY)
    xpos = xpos.flatten('F')
    ypos = ypos.flatten('F') 
    dz = durations.flatten() 
    minZ, maxZ = min(dz), max(dz)
    zpos = np.full_like(dz, minZ /2, dtype=np.double)   
     
    dx = 0.5 * np.ones_like(zpos)
    dy = dx.copy()  
    cmap = cm.get_cmap('jet')
    rgb = [cmap((k-minZ)/ (maxZ-minZ)) for k in dz] 
    ax.bar3d(xpos, ypos, zpos, 30, 30, dz, color=rgb, alpha=0.2, zsort='average')

    # Customize the z axis.  
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.03f'))

    ax.legend()   
    ax.set_zlim(minZ /2, maxZ * 2)
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel) 
    ax.set_zlabel("durée (en ms)")
    ax.invert_yaxis()
 

    d, x, y  = getBestParameters(durations, sizesX, sizesY) 
    plt.figtext(0.53, -0.03, "Meilleur temps: " + str(minZ) + " ms\navec " + xLabel + " : " + str(x) + "\net   " + xLabel + " : " + str(x) , ha="center", bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    plt.show()

 
showHistogramDuration3DHist(durations, sizes, sizes, "test", "taille x", "taille y")

NameError: name 'durations' is not defined

In [14]:
 

def testBlocKernels(kernels, nTry = 1):
    for i in range(len(kernels)):
        print("Execution du kernel : ", kernels[i], "avec ", nTry, " essaie.")

        durations, sizes = evaluate2DBlocSize(0, nTry)   
        showHistogramDuration3DHist(durations, sizes, sizes, kernels[i], "taille de bloc x", "taille de bloc y") 
 

getBestParameters(test, sizes, sizes)    
#testBlocKernels(KERNELS, 2)

(0.155969, 1, 1024)

In [ ]:


print("Variation de la taille des blocs histogram")
durations, sizes = evaluate2DBlocSize(3, 10)  
showHistogramDuration3D_1(durations, sizes, sizes, "taille x", "taille y")

In [ ]:
"""def showHistogramDurationtest(durations, sizes, title, xLabel, yLabel):
    plt.title(title)  
    plt.xlabel(xLabel) 
    plt.ylabel(yLabel) 
    for tab in durations :
        plt.plot(tab)
    plt.show()

showHistogramDurationtest(durations, sizes, "test", "nbthread", )"""